<a href="https://colab.research.google.com/github/aarongraybill/Math210/blob/main/ProblemSets/Midterm1/Midterm1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Math 210
## Aaron Graybill
### Problem Set 4
### 3/11/21

## Problem 1.
First let me bring in all of the predefined scripts and packages:

In [1]:
import numpy as np

In [2]:
def print_tableau(a,indep_names,dep_names):
#
# Given matrix "a" and lists of variables names "indep_names" and "dep_names",
# this function prints the matrix and labels in standard tableau format
# (including adding the -1, the minus signs in the last column, and labeling the lower-right as obj)
#
# First, check the inputs: indep_names should be one shorter than the number of columns of A
#                          dep_names should be one shorter than the number of rows of A
#
    nrows = a.shape[0]    # use the shape function to determine number of rows and cols in A
    ncols = a.shape[1]
    nindep = len(indep_names)
    ndep = len(dep_names)
    if nindep != ncols-1:
        print("WARNING: # of indep vbles should be one fewer than # columns of matrix")
    if ndep != nrows-1:
        print("WARNING: # of dep vbles should be one fewer than # rows of matrix")
# Now do the printing (uses a variety of formatting techniques in Python)        
    for j in range(ncols-1):                    # Print the independent variables in the first row
        print(indep_names[j].rjust(10),end="")  # rjust(10) makes fields 10 wide and right-justifies;
                                                #    the end command prevents newline)
    print("        -1")                         # Tack on the -1 at the end of the first row
    for i in range(nrows-1):
        for j in range(ncols):                  # Print all but the last row of the matrix
            print("%10.3f" % a[i][j],end="") # The syntax prints in a field 10 wide, showing 3 decimal points
        lab = "= -" + dep_names[i]
        print(lab.rjust(10))
    for j in range(ncols):
        print("%10.3f" % a[nrows-1][j],end="")  # Print the last row of the matrix, with label "obj" at end
    lab = "= obj"
    print(lab.rjust(10))
    print(" ")    # Put blank line at bottom

In [3]:
def pivot(a,pivrow,pivcol,indep_names,dep_names) :
# 
# Given matrix "a", a row number "pivrow" and column number "pivcol", 
#  and lists of variable names "indep_names" and "dep_names", this
#  function does three things:
#    (1) outputs the new version of the matrix after a pivot,
#    (2) updates the lists of variable names post-pivot
#    (3) prints the new matrix, including labels showing the variable names
#
# First, check the inputs: indep_names should be one shorter than the number of columns of A
#                          dep_names should be one shorter than the number of rows of A
#                          you should not be pivoting on the last row or last column
#
    a = a.astype(float)   # make sure entries are treated as floating point numbers
    nrows = a.shape[0]    # use the shape function to determine number of rows and cols in A
    ncols = a.shape[1]
    nindep = len(indep_names)
    ndep = len(dep_names)
    if nindep != ncols-1:
        print("WARNING: # of indep vbles should be one fewer than # columns of matrix")
    if ndep != nrows-1:
        print("WARNING: # of dep vbles should be one fewer than # rows of matrix")
    if pivrow > nrows-1 or pivcol > ncols-1:
        print("WARNING: should not pivot on last row or column")
    newa = a.copy()       # make a copy of A, to be filled in below with result of pivot
    p = a[pivrow-1][pivcol-1]   # identify pivot element
    newa[pivrow-1][pivcol-1] = 1/p   # set new value of pivot element
    # Set entries in p's row
    for j in range(ncols):
        if j != pivcol-1:
            newa[pivrow-1][j]=a[pivrow-1][j]/p;
    # Set entries in p's column
    for i in range(nrows):
        if i != pivrow-1:
            newa[i][pivcol-1]=-a[i][pivcol-1]/p;
    # Set all other entries
    for i in range(nrows):
        for j in range(ncols):
            if i != pivrow-1 and j != pivcol-1:
                r = a[i][pivcol-1]
                q = a[pivrow-1][j]
                s = a[i][j]
                newa[i][j]=(p*s-q*r)/p
    # Now swap the variable names
    temp = indep_names[pivcol-1]
    indep_names[pivcol-1]=dep_names[pivrow-1]
    dep_names[pivrow-1]=temp
    print_tableau(newa,indep_names,dep_names) # Print the matrix with updated labels
    return newa;

In [4]:
def target(a) :
    nrows = a.shape[0]    # use the shape function to determine number of rows and cols in "a"
    ncols = a.shape[1]
    import numpy as np
    v = np.empty(ncols-1)
    for i in range(ncols-1):
        v[i]=a[nrows-1,i]
    biggest_c = np.max(v)
    where_is_biggest_c = np.argmax(v)+1
    if biggest_c > 0 :
        return where_is_biggest_c
    else :
        return -1

In [5]:
def select(a,pivcolnum) :
    nrows = a.shape[0]    # use the shape function to determine number of rows and cols in A
    ncols = a.shape[1]
# First task: work down the column and record the b/a ratios in a vector v
#     except record -1 if a is negative or zero
    import numpy as np
    v = np.zeros(nrows-1)
    for i in range(nrows-1):
        if a[i,pivcolnum-1]>0 :
            v[i] = a[i,ncols-1]/a[i,pivcolnum-1]
        else :
            v[i] = -1
# Second task: if max b/a > -1, find min b/a by hand (ignoring zero entries in v)
    if np.max(v) > -1 :  
        min_so_far = np.max(v)+1  # Initialize variable to be for-sure bigger than the min
        for i in range(nrows-1):
            if v[i] > -1 and v[i] < min_so_far :
                min_so_far = v[i]
                where_is_min = i+1   # Add 1 to use human numbering
        return where_is_min      # Once we've scanned v for min, we can return result
    else :          
        return -1

In [6]:
# Create Simplex BF
def SimplexBF(a,indep_names,dep_names):
  nrows, ncols = a.shape
  a_new = a
  print_tableau(a_new,indep_names,dep_names)
  while np.max(a_new[nrows-1,:-1])>0:
    pivcol=target(a_new)
    pivrow=select(a_new,pivcol)
    if pivrow == -1:
      return("Unbounded")
    else:
      a_new=pivot(a_new,pivrow,pivcol,indep_names,dep_names)
      print_tableau(a_new,indep_names,dep_names)



### a.
Okay now I can start on SimplexNBF:

In [7]:
def targetnbf(tab):
  nrows, ncols = tab.shape
  new_i = -1
  for i in range(nrows-1) :# don't check obj fn row
    if tab[i,ncols-1] < 0:
      new_i = i+1
  return(new_i)

### b.

In [8]:
def candidateone(tab,targetedrow):
  #don't specify a row that's the obj fn row, I have no protocol against it
  nrows, ncols = tab.shape
  for i in range(ncols-1): #don't check last col bc it's the b's
    if tab[targetedrow-1,i]<0 :
      return(i+1)#bailout if found one
  return(-1) #none found in loop, return -1


### c.

In [9]:
def  selectnbf(tab,targetedrow,pivcolumn) :
  nrows, ncols = tab.shape

  #subset of two columns in question exclude obj fn row
  candidate_column =tab[targetedrow-1:nrows-1,pivcolumn-1]
  b_column=tab[targetedrow-1:nrows-1,ncols-1]

  # compute ratios
  ratios=b_column/candidate_column

  # Find row in subset and convert back to index in full table:
  #don't need to start at first row, let those be the starting values
  cur_ratio= ratios[0]
  best_row=0
  # the seemingly misplaced -1s and +1s are because we start at second row
  for i in range(len(candidate_column)-1):
    if candidate_column[i+1]>0 and ratios[i+1]<cur_ratio :
      cur_ratio=ratios[i+1]
      best_row=i+1

  #the targetedrow already has the +1, so this converts to start at 1
    #and converts table subset index to full tableau index
  best_row=best_row+targetedrow  

  return([best_row,pivcolumn]) #output

### d.

In [34]:
def simplexnbf(tab,indep_names,dep_names):
  nrows, ncols = tab.shape #get dims
  current_target_row=np.Inf #set current target row to nonsense
  tab_new=tab # get primer value for tableau iterator
  
  NotReadyForBF = True #changed when problem is BF
  SolutionPossible = True #false when no solution is discovered

  print_tableau(tab,indep_names,dep_names) #print initial tableau

  # while NBF and solution still possible apply NBF rules
  while NotReadyForBF and SolutionPossible:
    current_target_row=targetnbf(tab_new) # find target
    if current_target_row ==-1 : #if target is -1, Ready for BF
      NotReadyForBF = False
    else : #find candidate from targeted row
      current_candidate_column=candidateone(tab_new,current_target_row)
      if current_candidate_column ==-1 : #if no candidates, no solution
        SolutionPossible = False
      else : #pivot based off of the computed selection
        pivot_row,pivot_col=selectnbf(tab_new,current_target_row,current_candidate_column)
        tab_new=pivot(tab_new,pivot_row,pivot_col,indep_names,dep_names)
  if not NotReadyForBF : #if we bailed because ready for BF apply SimplexBF
    SimplexBF(tab_new,indep_names,dep_names)
  if not SolutionPossible: # if we bailed bc constraint set emtpy, say so 
    return("-1, Constraint set empty, sorry ☹️")
  
  #we can't exit the while loop for anything but the reasons above, 
  # so don't need any else statements
  


That's actually not what the question is asking for, so let me tighten it up to give the requested output (though you can gather the desired result from the above).

In [53]:
def simplexnbf_final(tab,indep_names,dep_names):
  output = simplexnbf(tab,indep_names,dep_names)
  # my sad error message is a string, so when we see that, return -1
  if output=="-1, Constraint set empty, sorry ☹️": 
    return -1
  #else we must have a tableau for BF, return 0
  else : 
    return 0

In [54]:
a=np.array([
            [-2,-10,3,-20,8],
            [-1,1,0,-3,-6],
            [1,4,-1,8,-1],
            [1,1,1,1,0]])
print(a)
#print(a.shape)
print(targetnbf(a))
print(candidateone(a,2))
print(selectnbf(a,2,3))
indep_names=["x","y","z","w"]
dep_names=["t1","t2","t3"]
simplexnbf(a,indep_names,dep_names)
simplexnbf_final(a,indep_names,dep_names)

[[ -2 -10   3 -20   8]
 [ -1   1   0  -3  -6]
 [  1   4  -1   8  -1]
 [  1   1   1   1   0]]
3
1
[2, 3]
         x         y         z         w        -1
    -2.000   -10.000     3.000   -20.000     8.000     = -t1
    -1.000     1.000     0.000    -3.000    -6.000     = -t2
     1.000     4.000    -1.000     8.000    -1.000     = -t3
     1.000     1.000     1.000     1.000     0.000     = obj
 
         x         y        t3         w        -1
     1.000     2.000     3.000     4.000     5.000     = -t1
    -1.000     1.000     0.000    -3.000    -6.000     = -t2
    -1.000    -4.000    -1.000    -8.000     1.000      = -z
     2.000     5.000     1.000     9.000    -1.000     = obj
 
        t2         y        t3         w        -1
     1.000     3.000     3.000     1.000    -1.000     = -t1
    -1.000    -1.000    -0.000     3.000     6.000      = -x
    -1.000    -5.000    -1.000    -5.000     7.000      = -z
     2.000     7.000     1.000     3.000   -13.000     = obj
 
     

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: RuntimeWarning: divide by zero encountered in true_divide
  if __name__ == '__main__':


-1

In [55]:
a= -1*np.array([
    [1,1,0,0,0,0,0,0,0,15],
    [0,0,2,2,1,1,0,0,0,50],
    [0,0,0,0,0,0,2,2,0,50],
    [0,0,0,0,0,0,1,0,2,40],
    [0,0,0,0,2,2,0,0,1,40],
    [2,4,2,4,2,4,0,0,4,245],
    [1,0,1,0,1,0,1,4,0,100],
    [1,1,1,1,1,1,1,1,1,0]
  ])

indep_names = ["p1","p2","p3","p4","p5","p6","p7","p8","p9"]

dep_names = ["4x6","2x6","2x5","3x3","1x6","1x2","1x3"]

simplexnbf(a,indep_names,dep_names)
simplexnbf_final(a,indep_names,dep_names)


        p1        p2        p3        p4        p5        p6        p7        p8        p9        -1
    -1.000    -1.000     0.000     0.000     0.000     0.000     0.000     0.000     0.000   -15.000    = -4x6
     0.000     0.000    -2.000    -2.000    -1.000    -1.000     0.000     0.000     0.000   -50.000    = -2x6
     0.000     0.000     0.000     0.000     0.000     0.000    -2.000    -2.000     0.000   -50.000    = -2x5
     0.000     0.000     0.000     0.000     0.000     0.000    -1.000     0.000    -2.000   -40.000    = -3x3
     0.000     0.000     0.000     0.000    -2.000    -2.000     0.000     0.000    -1.000   -40.000    = -1x6
    -2.000    -4.000    -2.000    -4.000    -2.000    -4.000     0.000     0.000    -4.000  -245.000    = -1x2
    -1.000     0.000    -1.000     0.000    -1.000     0.000    -1.000    -4.000     0.000  -100.000    = -1x3
    -1.000    -1.000    -1.000    -1.000    -1.000    -1.000    -1.000    -1.000    -1.000     0.000     = obj
 
       1x

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: RuntimeWarning: divide by zero encountered in true_divide
  if __name__ == '__main__':


    72.500     = -p6
     0.000     0.000    -0.500     1.000     0.250     0.000     0.000     0.000    -0.250    15.000    = -2x6
    -0.000    -0.000    -0.000    -0.000    -0.000    -0.000    -2.000    -2.000     4.000    30.000     = -p9
    -0.000    -0.000     0.000    -0.000    -0.000    -0.000    -1.000    -0.000     2.000    40.000    = -3x3
    -0.000    -0.000    -0.000    -0.000    -0.500     1.000     0.000    -0.000     0.500    20.000    = -1x6
    -0.500     2.000    -0.000     2.000     0.000     2.000    -1.000    -4.000     4.000    62.500     = -p8
    -0.500     2.000     0.500     1.000     0.250     1.000    -0.000    -0.000     1.750    87.500    = -1x2
    -0.500     1.000    -0.000     1.000    -0.000     1.000    -1.000    -1.000     3.000   162.500     = obj
 
        p2       1x3        p4        p3        p5       4x6        p7        p1        p9        -1
    -0.500     1.000     0.500     1.000     0.250     1.000     0.875     0.875    -0.438    59.37

0

In [38]:

new_constraints=np.append(np.identity(9),np.transpose(np.array([np.repeat(15,9)])),axis=1)

b=np.append(new_constraints,a,axis=0)

indep_names = ["p1","p2","p3","p4","p5","p6","p7","p8","p9"]

dep_names = ["p1_15","p2_15","p3_15","p4_15","p5_15","p6_15","p7_15","p8_15","p9_15","4x6","2x6","2x5","3x3","1x6","1x2","1x3"]

#print_tableau(b,indep_names,dep_names)

simplexnbf(b,indep_names,dep_names)

        p1        p2        p3        p4        p5        p6        p7        p8        p9        -1
     1.000     0.000     0.000     0.000     0.000     0.000     0.000     0.000     0.000    15.000  = -p1_15
     0.000     1.000     0.000     0.000     0.000     0.000     0.000     0.000     0.000    15.000  = -p2_15
     0.000     0.000     1.000     0.000     0.000     0.000     0.000     0.000     0.000    15.000  = -p3_15
     0.000     0.000     0.000     1.000     0.000     0.000     0.000     0.000     0.000    15.000  = -p4_15
     0.000     0.000     0.000     0.000     1.000     0.000     0.000     0.000     0.000    15.000  = -p5_15
     0.000     0.000     0.000     0.000     0.000     1.000     0.000     0.000     0.000    15.000  = -p6_15
     0.000     0.000     0.000     0.000     0.000     0.000     1.000     0.000     0.000    15.000  = -p7_15
     0.000     0.000     0.000     0.000     0.000     0.000     0.000     1.000     0.000    15.000  = -p8_15
     0.000 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: RuntimeWarning: divide by zero encountered in true_divide
  if __name__ == '__main__':


    -0.250    -0.000     1.250  = -p5_15
    -0.000     0.000     0.000     0.000    -0.000     1.000     0.000    -0.000    -0.000    15.000  = -p6_15
     0.000     0.000    -0.000     0.000     0.000    -0.000    -0.000    -1.000    -0.500    10.000     = -p8
    -0.000     0.000     0.000     0.000    -0.000     0.000     0.000     1.000     0.500     5.000  = -p8_15
    -0.000     0.000     0.000     0.000    -0.000     0.000     0.500     0.500    -0.000     2.500  = -p9_15
     0.000     1.000    -1.000    -1.000     0.500    -1.000    -0.250     2.750     2.000    16.250    = -4x6
    -0.000     0.000     0.000     1.000    -0.000     0.000     0.000    -0.000    -0.000    15.000     = -p3
    -0.000     0.000     0.000     0.000    -0.000     0.000    -0.500    -0.500    -0.000    12.500     = -p9
    -0.000     0.000     0.000     0.000    -0.000     0.000     0.000     1.000    -0.000    15.000     = -p7
    -0.000     0.000     0.000     0.000    -0.500     1.000     0.250 